In [ ]:
#install editable version of this repository
# !python -m pip install -e .

In [1]:
import os 
import sys
from importlib import reload

#imports 
import torch
from torch.cuda import memory_allocated, empty_cache
from datasets import load_dataset
from torch.utils.data import DataLoader

# Import local functions ==================================
import activation_extractor
# from activation_extractor import *
# reload(activation_extractor)

In [2]:
# from activation_extractor.extractors import intermediateExtractorBase 
# reload(intermediateExtractorBase)
# from activation_extractor.inferencers import inferencerBase
# reload(inferencerBase)
from activation_extractor.model_functions import load_models, tokenize_funs, inference_funs, default_hooked_layers
reload(load_models)
reload(tokenize_funs)
reload(inference_funs)
reload(default_hooked_layers)

# from activation_extractor.model_functions import embedding_to_numpy
# reload(embedding_to_numpy)
# from activation_extractor.model_functions.embedding_to_numpy import *
reload(activation_extractor)

<module 'activation_extractor' from '/orfeo/cephfs/scratch/area/evillegas/glm/activation-extractor/src/activation_extractor/__init__.py'>

# Proteins/DNA

In [ ]:
load_dataset("ILSVRC/imagenet-1k")

In [ ]:
model_name = "facebook/esm2_t48_15B_UR50D"
inferencer = activation_extractor.Inferencer(model_name, device=None, half=False)

In [ ]:
l1, l2, l3 = [], [], []
for i in range(0,32):
    l1.append(inferencer.model.blocks[i].mlp.l1.weight.norm().detach().cpu().to(torch.float32).numpy())
    l2.append(inferencer.model.blocks[i].mlp.l2.weight.norm().detach().cpu().to(torch.float32).numpy())
    l3.append(inferencer.model.blocks[i].mlp.l3.weight.norm().detach().cpu().to(torch.float32).numpy())
    
import matplotlib.pyplot as plt
plt.plot(l1)
plt.plot(l2)
plt.plot(l3)

In [ ]:
# sequences = ["AAAAA", "PPPPPP"]
sequences = ["AAAA"]

In [ ]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(sequences) #tokenize
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [ ]:
extractor.get_outputs()

In [ ]:
dir = 

# Images 

In [ ]:
from PIL import Image
import requests

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)

In [ ]:
model_name = "timm/vgg16.tv_in1k"
inferencer = activation_extractor.Inferencer(model_name, device='cuda', half=False)

In [ ]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type)
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(sequences) #tokenize
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [ ]:
extractor.get_outputs()

# Text

In [ ]:
# !pip install git+https://github.com/HazyResearch/flash-attention.git

In [10]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import requests
from datasets import load_dataset

In [11]:
ds = load_dataset("ChristophSchuhmann/MS_COCO_2017_URL_TEXT")
text = ds['train']['TEXT'][0]

In [12]:
model_name = "togethercomputer/StripedHyena-Nous-7B"
inferencer = activation_extractor.Inferencer(model_name, device="cuda", half=False)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type, modality="sequence")
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(text) #tokenize
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

Initializing inference params...


In [15]:
seq_length=processed['attention_mask'].reshape(-1).shape[0]-1
extractor.save_outputs('test', output_id=1, 
                       emb_formats=["custom"], custom_position=seq_length)

backbone.embedding_layer not in dictionary


# Multimodal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

import requests
from datasets import load_dataset, Dataset

import pandas as pd

In [ ]:
ds = load_dataset("ChristophSchuhmann/MS_COCO_2017_URL_TEXT")

In [ ]:
idx = 501

# Open the image file
#online
url = ds['train']['URL'][idx]
# image = Image.open(requests.get(url, stream=True).raw)
# image = np.array(image)

#locally
local_path =  "../data/train2017/"
image_path = ds['train']['URL'][idx].split("/")[-1]
# image_path = dataset['URL'][idx].split("/")[-1]
image_path = os.path.join(local_path,image_path)
img = Image.open(image_path)
img_array = np.array(img)

if len(img_array.shape)==2:
    img_array =  np.stack((img_array,) * 3, axis=-1)
    
plt.imshow(img_array)

#text
text = ds['train']['TEXT'][idx]
print(text)

#inputs
input_data = {"text":text,
             "image":img_array
             }


In [ ]:
model_name = "openai/clip-vit-base-patch32"
inferencer = activation_extractor.Inferencer(model_name, device='cpu', half=False)

In [ ]:
#intermediate activation extractor
layers_to_hook = activation_extractor.get_layers_to_hook(inferencer.model,inferencer.model_type, modality="image-text")
extractor = activation_extractor.IntermediateExtractor(inferencer.model, layers_to_hook)
extractor.register_hooks()

#inference
processed = inferencer.process(input_data) #tokenize
outputs = inferencer.inference(processed)

#extractor outputs
#extractor.save_outputs('results/embeddings/test')
extractor.clear_all_hooks()

In [ ]:
extractor.get_outputs()

# Inference Over a Dataset

## Bio

In [ ]:
import numpy as np
from activation_extractor.scripts.inference import main_inference, load_the_data

In [ ]:
model_name="Rostlab/ProstT5"
output_folder="test"
emb_format="mean"
save_method="numpy"
max_batches=1

data_args = {
        "data_type":"protein",
        "batch_size":8,
        "data_source":"huggingface",
        "dataset_name":"proteinea/remote_homology",
        "dataset_partition":"train",
        "target_col":"primary",
        # "data_source":"local",
        # "target_col":"protein_seq",
        # "input_path":"/orfeo/LTS/LADE/LT_storage/bio_data/NCBI/NCBI_GP_from_geneid_Plt1000_Glt6000.csv",
        "max_length":1000,
    }


In [ ]:
data_loader = load_the_data(**data_args)
batch = next(iter(data_loader))
batch

In [ ]:
main_inference(model_name, output_folder, emb_format, save_method, max_batches, data_args)

## For Images/text

In [3]:
import numpy as np
import pandas as pd
from PIL import Image

from activation_extractor.scripts.inference import main_inference, load_the_data

from datasets import load_dataset, Dataset

In [4]:
ds = load_dataset("ChristophSchuhmann/MS_COCO_2017_URL_TEXT")
ds = pd.DataFrame(ds['train'])
ds.drop_duplicates(subset=['URL'])
ds = Dataset.from_pandas(ds)

In [5]:
model_name="togethercomputer/StripedHyena-Nous-7B"
# model_name="google/vit-base-patch16-224-in21k"
output_folder="test"
emb_format="mean"
save_method="numpy"
max_batches=1

data_args = {
        "data_type":"mscoco",
        # "modality":"image-text",
        "modality":"text",
        # "modality": "image",
        "batch_size":1,
        "data_source":"huggingface",
        "dataset_name":"ChristophSchuhmann/MS_COCO_2017_URL_TEXT",
        "dataset_partition":"train",
        # "data_source":"local",
        # "input_path":"test.csv",
        "image_source":"local",
        "image_dir":"../data/train2017",
    }

save_args = {
        "save_method":"numpy",
        "emb_formats":['LT','FT','mean'],
        "sequence_axis": 1,
}

In [6]:
data_loader = load_the_data(**data_args)
input_data = next(iter(data_loader))

In [7]:
main_inference(model_name, output_folder, save_args, max_batches, data_args)
        print(output_folder)
        print(output_id)
        print(reset)
        print(move_to_cpu)
        print(save_method)
        print(emb_formats)
        print(sequence_axis)
        print(custom_position)

/orfeo/cephfs/scratch/area/evillegas/glm/dgxtorch/lib/python3.10/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Output folder is: test/togethercomputer/StripedHyena-Nous-7B
Initializing inference params...


✔️ togethercomputer/StripedHyena-Nous-7B


test/togethercomputer/StripedHyena-Nous-7B
0
False
True
numpy
['LT', 'FT', 'mean']
1
None


AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
def collate_pil(batch):
    """
    Convert PIL images to numpy arrays.
    """
    batch = [np.array(img) for img in batch]

In [ ]:
ds = [ np.array(dataset["train"]["img"][i]) for i in range(10) ]

In [ ]:
ds[0].shape

In [ ]:
from torchvision.datasets import CIFAR100
import torchvision.transforms as transforms

In [ ]:
# Define the transformations
transform = transforms.Compose([
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761))  # Normalize the images
])

train_dataset = CIFAR100(
    root='./data',  # Directory to store the dataset
    train=True,  # Download the training dataset
    download=True,  # Download if not already downloaded
    transform=transform  # Apply transformations
)

train_loader = DataLoader(
    train_dataset,
    batch_size=64,  # Number of samples per batch
    shuffle=True,  # Shuffle the dataset
    num_workers=2,  # Number of subprocesses to use for data loadingt
)

In [ ]:
data_loader = DataLoader(train_dataset, batch_size=2, 
                             shuffle=False, collate_fn=None)

In [ ]:
from datasets import load_dataset
from torch.utils.data import DataLoader
import numpy as np

dataset = load_dataset("uoft-cs/cifar100")
data_loader = DataLoader(dataset["train"], batch_size=2, 
                             shuffle=False, collate_fn=collate_pil)